In [1]:
import os
import pandas as pd 
#import modin.pandas as pd # emails went to 236
from tqdm import tqdm
import datetime
from p_tqdm import p_map
from functools import partial
import gc

In [2]:
time_resolution = '1week'
lk_path = '/mnt/data0/lkyin/'
c_path = './network_data'+str(time_resolution)+'/commits/'
e_path = './network_data'+str(time_resolution)+'/emails/'
mix_path = './network_data'+str(time_resolution)+'/mix/'

## Processing commits data

In [3]:
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = './weekly_data/commits/'
if not os.path.exists(to_path):
	os.makedirs(to_path)


reading commits...


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



grouping by project...


In [5]:
df.keys()

dict_keys(['abdera', 'accumulo', 'ace', 'activemq', 'adffaces', 'agila', 'airavata', 'airflow', 'allura', 'alois', 'amaterasu', 'ambari', 'ant', 'any23', 'apex', 'apisix', 'ariatosca', 'aries', 'asterixdb', 'atlas', 'aurora', 'awf', 'batchee', 'beam', 'beehive', 'bigtop', 'bloodhound', 'bluesky', 'blur', 'bookkeeper', 'brooklyn', 'buildr', 'bval', 'calcite', 'carbondata', 'cassandra', 'cayenne', 'celix', 'chemistry', 'chukwa', 'clerezza', 'click', 'climate', 'cloudstack', 'cmda', 'commons', 'commonsrdf', 'composer', 'concerted', 'cordova', 'corinthia', 'couchdb', 'crunch', 'ctakes', 'curator', 'cxf', 'daffodil', 'datafu', 'datasketches', 'deltacloud', 'deltaspike', 'derby', 'devicemap', 'directmemory', 'directory', 'dolphinscheduler', 'drill', 'droids', 'druid', 'dubbo', 'eagle', 'echarts', 'edgent', 'empire', 'esme', 'etch', 'falcon', 'felix', 'fineract', 'flex', 'flink', 'flume', 'fluo', 'freemarker', 'ftpserver', 'gearpump', 'geode', 'giraph', 'gobblin', 'gora', 'gossip', 'graffito'

In [4]:
df_data.columns

Index(['project_name', 'list_name', 'date', 'month', 'message_id',
       'sender_name', 'sender_email', 'author_name', 'author_email',
       'file_name', 'loc', 'ref_or_sha', 'subject', 'commit_type',
       'incubation_time', 'author_full_name', 'is_bot', 'is_coding',
       'dealised_author_full_name'],
      dtype='object')

In [5]:
df_data[['date','month']].head(10)

,date,month
0,2009-01-12 17:11:30,4
1,2009-01-12 17:11:30,4
2,2009-01-12 17:11:30,4
3,2009-01-12 17:11:30,4
4,2009-01-12 17:11:30,4
5,2009-01-12 17:11:30,4
6,2009-01-12 17:11:30,4
7,2009-01-12 17:11:30,4
8,2009-01-12 17:11:30,4
9,2009-01-12 17:11:30,4


In [6]:
df_data['date'].describe

<bound method Series.describe of 0          2009-01-12 17:11:30
1          2009-01-12 17:11:30
2          2009-01-12 17:11:30
3          2009-01-12 17:11:30
4          2009-01-12 17:11:30
                  ...         
4312472    2013-01-25 12:55:09
4312473    2013-01-25 12:55:10
4312474    2013-01-25 12:55:10
4312475    2013-01-25 12:55:10
4312476    2013-01-25 12:55:10
Name: date, Length: 4312477, dtype: object>

In [7]:
df_data = df_data.loc[df_data.date.str.len()==19]
len(df_data)

4312196

In [8]:
df_data['datetime'] = df_data['date'].apply(lambda x: pd.Period(x))

In [10]:
df_data['datetime'].describe

<bound method Series.describe of 0          2009-01-12 17:11:30
1          2009-01-12 17:11:30
2          2009-01-12 17:11:30
3          2009-01-12 17:11:30
4          2009-01-12 17:11:30
                  ...         
4312472    2013-01-25 12:55:09
4312473    2013-01-25 12:55:10
4312474    2013-01-25 12:55:10
4312475    2013-01-25 12:55:10
4312476    2013-01-25 12:55:10
Name: datetime, Length: 4312196, dtype: period[S]>

In [13]:
df_data['year'] = df_data['datetime'].apply(lambda x: x.year)

In [14]:
df_data['dayofyear'] = df_data['datetime'].apply(lambda x: x.dayofyear)

In [15]:
df_data['weekofyear'] = df_data['datetime'].apply(lambda x: x.weekofyear)

In [16]:
df_data['week'] = df_data['datetime'].apply(lambda x: pd.Period(x, freq='W'))

In [17]:
df_data['week'].describe()

count                   4312196
unique                      834
top       2011-05-30/2011-06-05
freq                      69047
Name: week, dtype: object

In [3]:
def save_each_week(week, weekly_df_dict, weekint, to_path):
	weekly_df = weekly_df_dict[week]
	weekly_df = weekly_df[weekly_df['dealised_author_full_name'].notna()]
	if weekly_df.empty: 
		return
	weekstr = str(weekint).zfill(3)
	file_path = to_path + '{}__{}.csv'.format(project, weekstr)
	weekly_df.to_csv(file_path, index=False)
	gc.collect()
	return

In [ ]:
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['week'] = df_data['date'].apply(lambda x: pd.Period(x, freq='W-MON'))
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = './weekly_data/commits/'
if not os.path.exists(to_path):
	os.makedirs(to_path)


print('grouping by period...')
for project in tqdm(df):
	weekly_df_dict_data = dict(tuple(df[project].groupby(df[project]['week'])))
	listkeys = list(weekly_df_dict.keys())
	listkeys.sort()
	p_map(partial(save_each_week, weekly_df_dict= weekly_df_dict, to_path=to_path), listkeys, range(len(listkeys)), num_cpus = 4)
	gc.collect()
print('Commits Done.')

In [5]:
listkeys = list(weekly_df_dict.keys())
listkeys.sort()
print(listkeys)
len(listkeys)

[Period('2006-06-06/2006-06-12', 'W-MON'), Period('2006-06-13/2006-06-19', 'W-MON'), Period('2006-06-20/2006-06-26', 'W-MON'), Period('2006-06-27/2006-07-03', 'W-MON'), Period('2006-07-04/2006-07-10', 'W-MON'), Period('2006-07-11/2006-07-17', 'W-MON'), Period('2006-07-18/2006-07-24', 'W-MON'), Period('2006-07-25/2006-07-31', 'W-MON'), Period('2006-08-01/2006-08-07', 'W-MON'), Period('2006-08-08/2006-08-14', 'W-MON'), Period('2006-08-15/2006-08-21', 'W-MON'), Period('2006-08-22/2006-08-28', 'W-MON'), Period('2006-08-29/2006-09-04', 'W-MON'), Period('2006-09-05/2006-09-11', 'W-MON'), Period('2006-09-12/2006-09-18', 'W-MON'), Period('2006-09-19/2006-09-25', 'W-MON'), Period('2006-09-26/2006-10-02', 'W-MON'), Period('2006-10-03/2006-10-09', 'W-MON'), Period('2006-10-10/2006-10-16', 'W-MON'), Period('2006-10-17/2006-10-23', 'W-MON'), Period('2006-10-24/2006-10-30', 'W-MON'), Period('2006-10-31/2006-11-06', 'W-MON'), Period('2006-11-07/2006-11-13', 'W-MON'), Period('2006-11-14/2006-11-20', '

110

In [14]:
len(weekly_df_dict.keys())

107

In [ ]:
# ------------------- processing commits ---------------------- 
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['week'] = df_data['date'].apply(lambda x: pd.Period(x, freq='W-MON'))
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = './weekly_data/commits/'
if not os.path.exists(to_path):
	os.makedirs(to_path)


print('grouping by period...')
for project in tqdm(df):
	weekly_df_dict = dict(tuple(df[project].groupby(df[project]['week'])))
	weekint = 0
	for week in weekly_df_dict:
		weekly_df = weekly_df_dict[week]
		weekly_df = weekly_df[weekly_df['dealised_author_full_name'].notna()]
		weekint+=1
		if weekly_df.empty: continue
		weekstr = str(weekint).zfill(3)
		file_path = to_path + '{}__{}.csv'.format(project, weekstr)
		weekly_df.to_csv(file_path, index=False)
print('Commits Done.')




In [ ]:
# ------------------- processing emails ---------------------- 
print('reading emails...')
df_data = pd.read_csv(lk_path+'emails_final.csv')
print('grouping by project...')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['week'] = df_data['date'].apply(lambda x: pd.Period(x, freq='W-MON'))
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = './weekly_data/emails/'
if not os.path.exists(to_path):
	os.makedirs(to_path)

print('grouping by period...')
for project in tqdm(df):
	weekly_df_dict = dict(tuple(df[project].groupby(df[project]['week'])))
	weekint = 0
	for week in weekly_df_dict:
		weekly_df = weekly_df_dict[week]
		weekly_df = weekly_df[weekly_df['dealised_author_full_name'].notna()]
		weekint+=1
		if weekly_df.empty: continue
		weekstr = str(weekint).zfill(3)
		file_path = to_path + '{}__{}.csv'.format(project, weekstr)
		weekly_df.to_csv(file_path, index=False)
print('Emails Done.')